In [9]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare

# Chi Square Test

I suspect the dice in my Settlers of Catan set are not fair dice. So I wrote down all the outcomes of two games and I'm going to do a Chi Square test for goodness of fit.

## Load the data

In [2]:
df = pd.read_csv('../data/catan_dice.csv')

In [3]:
df.head(1)

,Value,Count G1,Count G2
0,2,2,1


## Set index to roll outcomes

In [4]:
df = pd.DataFrame(df[['Count G1', 'Count G2']].values, index=df['Value'], columns=['G1', 'G2'])

In [5]:
df.head(1)

,G1,G2
Value,,
2,2,1


## Add a total observations column

In [6]:
def both_games(x):
    '''Use axis=1'''
    a, b = x
    return int(a) + int(b)

In [7]:
df['Both'] = df.apply(both_games, axis=1)

In [8]:
df.head(1)

,G1,G2,Both
Value,,,
2,2,1,3


## Find the expected outcomes

In [10]:
outcomes_dict = {}

In [11]:
for col in df.columns:
    outcomes_dict[col] = df[col].sum()

In [12]:
outcomes_dict

{'Both': 83, 'G1': 52, 'G2': 31}

In [29]:
rolls = [i for i in range(1,7)]
rolls

[1, 2, 3, 4, 5, 6]

In [30]:
from collections import defaultdict

In [31]:
expected = defaultdict(float)

In [32]:
for i in range(1,13):
    for j in rolls:
        for k in rolls:
            if j+k == i:
                expected[i] += 1/36

In [33]:
expected

defaultdict(float,
            {2: 0.027777777777777776,
             3: 0.05555555555555555,
             4: 0.08333333333333333,
             5: 0.1111111111111111,
             6: 0.1388888888888889,
             7: 0.16666666666666669,
             8: 0.1388888888888889,
             9: 0.1111111111111111,
             10: 0.08333333333333333,
             11: 0.05555555555555555,
             12: 0.027777777777777776})

In [37]:
e = np.array(list(expected.values()))
e

array([ 0.02777778,  0.05555556,  0.08333333,  0.11111111,  0.13888889,
        0.16666667,  0.13888889,  0.11111111,  0.08333333,  0.05555556,
        0.02777778])

In [40]:
for col in df.columns:
    # use outcomes_dict
    ex = outcomes_dict[col]*e
    df[f'e_{col}'] = ex
    print(ex, '\n')

[ 1.44444444  2.88888889  4.33333333  5.77777778  7.22222222  8.66666667
  7.22222222  5.77777778  4.33333333  2.88888889  1.44444444] 

[ 0.86111111  1.72222222  2.58333333  3.44444444  4.30555556  5.16666667
  4.30555556  3.44444444  2.58333333  1.72222222  0.86111111] 

[  2.30555556   4.61111111   6.91666667   9.22222222  11.52777778
  13.83333333  11.52777778   9.22222222   6.91666667   4.61111111
   2.30555556] 



In [41]:
df.head(1)

,G1,G2,Both,e_G1,e_G2,e_Both
Value,,,,,,
2,2,1,3,1.444444,0.861111,2.305556


## Chi Square test

    chisq, p = scipy.stats.chisquare(f_obs, f_exp=None, ddof=0, axis=0)

[Citation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html)

In [63]:
d = {
    'G1': [i for i in df.columns if 'G1' in i],
    'G2': [i for i in df.columns if 'G2' in i],
    'Both': [i for i in df.columns if 'Both' in i]
}
d

{'Both': ['Both', 'e_Both'], 'G1': ['G1', 'e_G1'], 'G2': ['G2', 'e_G2']}

In [81]:
for i, j in d.items():
    obs = df[j[0]].values
    exp = df[j[1]].values
    chi, p = chisquare(obs, exp, ddof=1)
    print(f'{"Group:": <20}{i}')
    print(f'{"Chisq:": <20}{chi}')
    print(f'{"P:": <20}{p}')
    print('Observed/Expected pairs:')
    pairs = [(i,j,k) for i, j, k in zip(df.index, obs, exp)]
    for i, j, k in pairs:
        print(f'{i: <3} | {j: <2} / {k:.2}')
    print(f'{"":->30}')

Group:              G1
Chisq:              9.903846153846153
P:                  0.3583254114540303
Observed/Expected pairs:
2   | 2  / 1.4
3   | 2  / 2.9
4   | 5  / 4.3
5   | 5  / 5.8
6   | 13 / 7.2
7   | 6  / 8.7
8   | 6  / 7.2
9   | 4  / 5.8
10  | 7  / 4.3
11  | 1  / 2.9
12  | 1  / 1.4
------------------------------
Group:              G2
Chisq:              22.283870967741937
P:                  0.008021552465671324
Observed/Expected pairs:
2   | 1  / 0.86
3   | 2  / 1.7
4   | 0  / 2.6
5   | 10 / 3.4
6   | 7  / 4.3
7   | 3  / 5.2
8   | 5  / 4.3
9   | 1  / 3.4
10  | 2  / 2.6
11  | 0  / 1.7
12  | 0  / 0.86
------------------------------
Group:              Both
Chisq:              18.50843373493976
P:                  0.02971258262498292
Observed/Expected pairs:
2   | 3  / 2.3
3   | 4  / 4.6
4   | 5  / 6.9
5   | 15 / 9.2
6   | 20 / 1.2e+01
7   | 9  / 1.4e+01
8   | 11 / 1.2e+01
9   | 5  / 9.2
10  | 9  / 6.9
11  | 1  / 4.6
12  | 1  / 2.3
------------------------------


## Trial 1

In [18]:
g = df.groupby(df.columns, axis=1)

In [26]:
for name, group in g:
    print(name)
    obs = group.values.T[0]
    x = exp[name]
    print(f'Expected: {x}')
    print(f'Observations: {obs}')
    c, p = chisquare(obs, x, ddof=1)
    print(f'Chisq: {c}')
    print(f'P: {p}')
    print(f'{"":->20}')

Both
Expected: 13.833333333333334
Observations: [ 3  4  5 15 20  9 11  5  9  1  1]
Chisq: 57.371485943775106
P: 4.289809063646293e-09
--------------------
G1
Expected: 8.666666666666666
Observations: [ 2  2  5  5 13  6  6  4  7  1  1]
Chisq: 33.56410256410256
P: 0.00010647586941053547
--------------------
G2
Expected: 5.166666666666667
Observations: [ 1  2  0 10  7  3  5  1  2  0  0]
Chisq: 32.18817204301075
P: 0.00018474996356524843
--------------------
